<a href="https://colab.research.google.com/github/bhu800/GCSBot/blob/master/trainedModels/v1.1/train_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/My Drive/GCSBot/GCSBot_FAQ_db_v1.0.json').read()
intents = json.loads(data_file)

In [4]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['options (response is yet to be written!)'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [''], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'No

In [5]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
print(words)
print(classes)
print(documents)

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'Why', 'should', 'I', 'join', 'IIT', 'Mandi', '?', 'What', "'s", 'it', 'like', 'to', 'attend', 'IIT', 'Mandi', '?', 'Why', 'IIT', 'Mandi', '?', 'what', 'is', 'it', 'like', 'to', 'attend', 'IIT', 'Mandi', '?', 'How', 'is', 'life', 'at', 'IIT', 'Mandi', '?', 'How', 'is', 'the', 'hostel', 'life', 'at', 'IIT', 'Mandi', '?', 'How', 'is', 'the', 'campus', 'life', 'at', 'IIT', 'Mandi', '?', 'How', 'is', 'collage', 'life', 'in', 'iit', 'mandi', '?', 'Is', 'there', 'any', 'special', '

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documnets")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

380 documnets
126 classes ['5WIP', 'Biotech_branch', 'CE_branch', 'CSE_branch', 'DSE_branch', 'EE_branch', 'EP_branch', 'Exodia', 'HnT', 'How_to_reach_IITMANDI', 'IITMANDI_for_girls', 'IITMandi_vs_other_new_IITs', 'IIT_Mandi_campus', 'MCM_scholarships', 'ME_branch', 'MTB', 'NSO', 'NSS', 'art_geeks', 'atm_and_bank_facilities', 'books', 'branch_change_criterion', 'branch_change_cutoff_cgpa', 'bus_schedule_IITMandi', 'campus_facilities', 'campus_for_freshers', 'canteens_facilities', 'certificates_and_documents', 'change_room', 'cinema_hall', 'class_commencement', 'club_audition', 'co_curricular', 'coding_culture', 'companies_for_placements', 'cultural_clubs', 'curriculum_Biotech', 'curriculum_CE', 'curriculum_CSE', 'curriculum_DSE', 'curriculum_EE', 'curriculum_EP', 'curriculum_ME', 'curriculum_overall', 'cycle', 'dance_club', 'debating_club', 'drama_club', 'e_cell', 'education_loan', 'elective_courses', 'faculties', 'family_memebers_stay', 'famous_alumni', 'fees', 'festival_holiday', 'fe

In [8]:
pickle.dump(words,open('/content/drive/My Drive/GCSBot/trainedModels/v1.1/words.pkl','wb'))
pickle.dump(classes,open('/content/drive/My Drive/GCSBot/trainedModels/v1.1/classes.pkl','wb'))

In [9]:
training = []
output_empty = [0]*len(classes)
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [ ]:
print(training)

In [10]:
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [11]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [12]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, callbacks=[callbacks])
model.save('/content/drive/My Drive/GCSBot/trainedModels/v1.1/chatbot_model_v1.1.1.h5', hist)

Epoch 1/200
36325/36328 [============================>.] - ETA: 0s - loss: 0.3091 - accuracy: 0.9313
Reached 90% accuracy so cancelling training!
36328/36328 [==============================] - 84s 2ms/step - loss: 0.3091 - accuracy: 0.9313
